In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from matplotlib import offsetbox
import seaborn as sns

from sklearn.decomposition import PCA
from skimage.feature import hog

from tqdm import tqdm
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential

In [ ]:
train_ds=pd.read_csv('../input/Kannada-MNIST/train.csv')
test_ds=pd.read_csv('../input/Kannada-MNIST/test.csv')

In [ ]:
y_train=train_ds['label']
x_train=train_ds.drop(['label'],axis=1)

id_test=test_ds['id']
x_test=test_ds.drop(['id'],axis=1)

In [ ]:
np.save('x_train.npy',x_train)
np.save('y_train.npy',y_train)

np.save('id_test.npy',id_test)
np.save('x_test.npy',x_test)

In [ ]:
x_train=np.load('x_train.npy')
y_train=np.load('y_train.npy')

In [ ]:
train_ds['label'].value_counts().plot(kind='barh',figsize=(7,7))

In [ ]:
fig=plt.figure(figsize=(7,7))
idx=[]

for i in range(25):
    idx.append(np.random.randint(len(y_train)))
    
for i in range(25):
    label=y_train[idx[i]]
    img=(x_train[idx[i]]).reshape(28,28)
    
    plt.subplot(5, 5, i+1)
    plt.imshow(img,cmap='gray')
    plt.title(label)
    plt.axis('off')

In [ ]:
# Scale and visualize the embedding vectors
def plot_embedding(X,y, title=None,size=(15,12)):
    x_min, x_max = np.min(X, 0), np.max(X, 0)
    X = (X - x_min) / (x_max - x_min)

    plt.figure(figsize=size)
    ax = plt.subplot(111)
    for i in range(X.shape[0]):
        plt.text(X[i, 0], X[i, 1], str(y[i]),
                 color=plt.cm.Set1(y[i] / 10.),
                 fontdict={'weight': 'bold', 'size': 9})

    if hasattr(offsetbox, 'AnnotationBbox'):
        # only print thumbnails with matplotlib > 1.0
        shown_images = np.array([[1., 1.]])  # just something big
        for i in range(X.shape[0]):
            dist = np.sum((X[i] - shown_images) ** 2, 1)
            if np.min(dist) < 4e-3:
                # don't show points that are too close
                continue
            shown_images = np.r_[shown_images, [X[i]]]
            imagebox = offsetbox.AnnotationBbox(
                offsetbox.OffsetImage(x_train[i].reshape(28,28), cmap=plt.cm.gray_r),
                X[i])
            ax.add_artist(imagebox)
    plt.xticks([]), plt.yticks([])
    if title is not None:
        plt.title(title)
    plt.savefig('tsne.png')

In [ ]:
fig=plt.figure(figsize=(7,7))
for i in range(25):
    label=y_train[idx[i]]
    img=(x_train[idx[i]]).reshape(28,28)
    fd, hog_image = hog(img, orientations=10, pixels_per_cell=(2,2),cells_per_block=(1,1), visualize=True)
    plt.subplot(5, 5, i+1)
    plt.imshow(hog_image,cmap='gray')
    plt.title(label)
    plt.axis('off')

In [ ]:
fig=plt.figure(figsize=(7,7))
for i in range(25):
    label=y_train[idx[i]]
    img=(x_train[idx[i]]).reshape(28,28)
    fd, hog_image = hog(img, orientations=8, pixels_per_cell=(4,4),cells_per_block=(1,1), visualize=True)
    plt.subplot(5, 5, i+1)
    plt.imshow(hog_image,cmap='gray')
    plt.title(label)
    plt.axis('off')

In [ ]:
x_train_hog=[]

for i in tqdm(range(len(x_train))):
    img=x_train[i].reshape(28,28)
    fd, hog_image = hog(img, orientations=8, pixels_per_cell=(4,4),cells_per_block=(1,1), visualize=True)
    x_train_hog.append(fd)

In [ ]:
X=np.array(x_train_hog)
Y=np.array(y_train)

In [ ]:
x_train,x_val,y_train,y_val=train_test_split(X,Y,test_size=0.2)

print(x_train.shape)
print(x_val.shape)

In [ ]:
clf=SVC()
clf.fit(x_train,y_train)

In [ ]:
clf.score(x_train,y_train)

In [ ]:
clf.score(x_val,y_val)

In [ ]:
x_test=np.load('x_test.npy')
test_id=np.load('id_test.npy')

In [ ]:
x_test_hog=[]

for i in tqdm(range(len(x_test))):
    img=x_test[i].reshape(28,28)
    fd, hog_image = hog(img, orientations=8, pixels_per_cell=(4,4),cells_per_block=(1,1), visualize=True)
    x_test_hog.append(fd)

In [ ]:
y_pred_test=clf.predict(x_test_hog)

In [ ]:
my_submission = pd.DataFrame({'id': test_id, 'label': y_pred_test})
my_submission.to_csv('submission.csv', index=False)